In [4]:
import keyring
import os
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
# https://tavily.com/   # search result
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY


# Set State

In [5]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

# ToolNode로 도구 노드 구축

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

# ToolNode가 무엇일까?

In [7]:
import json

from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""
    
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}
        
    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call['id'],
                )
            )
        return {"messages": outputs}

# LLM 챗봇 설정

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state['messages'])
    return {"messages": [result]}

# Build graphs

In [9]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

## 인터넷 검색이 필요한 질문

In [10]:
graph.invoke({"messages": {"role": "user", "content": "2024년 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='2024년 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='e9376ea4-a0be-4e16-a739-cac583dac5f1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Wybiv96gMQuUF00i8K19J5g', 'function': {'arguments': '{"query":"2024년 한국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c9fe7fcd-dfc9-44e5-a1f6-6b5942cb73c4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 한국 대통령'}, 'id': 'call_5Wybiv96gMQuUF00i8K19J5g', 'type': 'tool

## LLM이 답할 수 있는 질문

In [11]:
graph.invoke({"messages": {"role": "user", "content": "마이크로소프트가 어떤 회사야?"}})

{'messages': [HumanMessage(content='마이크로소프트가 어떤 회사야?', additional_kwargs={}, response_metadata={}, id='27557dfd-e1d1-48d2-9a0b-b8372068a3f3'),
  AIMessage(content='마이크로소프트(Microsoft)는 미국의 다국적 기술 회사로, 주로 소프트웨어, 하드웨어, 전자기기 및 서비스 분야에서 활동하고 있습니다. 1975년에 빌 게이츠(Bill Gates)와 폴 앨런(Paul Allen)에 의해 설립되었습니다. 마이크로소프트는 특히 다음과 같은 제품과 서비스를 제공합니다:\n\n1. **운영 체제**: Windows 운영 체제는 개인용 컴퓨터에서 가장 널리 사용되는 운영 체제 중 하나입니다.\n2. **사무 소프트웨어**: Microsoft Office 제품군(Word, Excel, PowerPoint 등)은 비즈니스 및 개인 사용에서 널리 사용됩니다.\n3. **클라우드 서비스**: Azure는 마이크로소프트의 클라우드 컴퓨팅 서비스로, 데이터 저장, 서버 호스팅, 머신 러닝 및 기타 서비스 제공에 사용됩니다.\n4. **하드웨어**: Surface 시리즈의 태블릿 및 노트북, Xbox 게임 콘솔 등 다양한 하드웨어 제품을 제공합니다.\n5. **게임**: Xbox 및 Xbox Live와 같은 게임 플랫폼을 통해 게임 산업에서도 큰 영향을 미치고 있습니다.\n\n마이크로소프트는 기술 혁신을 통해 개인과 기업의 생산성을 향상시키고, 전 세계적으로 많은 사용자들에게 서비스를 제공하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 288, 'prompt_tokens': 89, 'total_tokens': 377, 'completion_tokens_details': {'accepted_prediction_tokens